## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
#from config import g_key
g_key = "c334a28706460f469f090fde04b34da0"

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

# Add a variable for the files to load
city_data_to_load = os.path.join("..","Weather_Database","WeatherPy_Database.csv")

# Read the City Data
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Nikolskoye,RU,59.7035,30.7861,55.29,90,87,5.30,overcast clouds
1,1,Nizhniy Odes,RU,63.6445,54.8560,40.69,85,100,11.72,overcast clouds
2,2,Mitsamiouli,KM,-11.3847,43.2844,79.09,69,40,5.75,scattered clouds
3,3,Pisco,PE,-13.7000,-76.2167,66.02,77,40,9.22,scattered clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.31,70,84,18.70,light rain
5,5,Cockburn Town,TC,21.4612,-71.1419,85.96,66,40,12.66,scattered clouds
6,6,Kilindoni,TZ,-7.9139,39.6668,77.04,73,45,12.77,scattered clouds
7,7,New Norfolk,AU,-42.7826,147.0587,54.90,67,98,14.00,overcast clouds
8,8,Dikson,RU,73.5069,80.5464,48.94,77,98,17.29,overcast clouds
9,9,Mataura,NZ,-46.1927,168.8643,45.48,99,98,2.39,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 150


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Nikolskoye,RU,59.7035,30.7861,55.29,90,87,5.30,overcast clouds
2,2,Mitsamiouli,KM,-11.3847,43.2844,79.09,69,40,5.75,scattered clouds
3,3,Pisco,PE,-13.7000,-76.2167,66.02,77,40,9.22,scattered clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.31,70,84,18.70,light rain
5,5,Cockburn Town,TC,21.4612,-71.1419,85.96,66,40,12.66,scattered clouds
6,6,Kilindoni,TZ,-7.9139,39.6668,77.04,73,45,12.77,scattered clouds
7,7,New Norfolk,AU,-42.7826,147.0587,54.90,67,98,14.00,overcast clouds
10,10,Kruisfontein,ZA,-34.0033,24.7314,54.88,74,70,7.90,broken clouds
11,11,Cabo San Lucas,MX,22.8909,-109.9124,90.05,64,1,3.44,clear sky
12,12,San Jose,US,37.3394,-121.8950,85.91,56,75,0.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                587
City                   587
Country                581
Lat                    587
Lng                    587
Max Temp               587
Humidity               587
Cloudiness             587
Wind Speed             587
Current description    587
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.count()

City_ID                581
City                   581
Country                581
Lat                    581
Lng                    581
Max Temp               581
Humidity               581
Cloudiness             581
Wind Speed             581
Current description    581
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,55.29,overcast clouds,59.7035,30.7861,
2,Mitsamiouli,KM,79.09,scattered clouds,-11.3847,43.2844,
3,Pisco,PE,66.02,scattered clouds,-13.7000,-76.2167,
4,Atuona,PF,78.31,light rain,-9.8000,-139.0333,
5,Cockburn Town,TC,85.96,scattered clouds,21.4612,-71.1419,
6,Kilindoni,TZ,77.04,scattered clouds,-7.9139,39.6668,
7,New Norfolk,AU,54.90,overcast clouds,-42.7826,147.0587,
10,Kruisfontein,ZA,54.88,broken clouds,-34.0033,24.7314,
11,Cabo San Lucas,MX,90.05,clear sky,22.8909,-109.9124,
12,San Jose,US,85.91,broken clouds,37.3394,-121.8950,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
print(len(hotel_df))
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
print(len(clean_hotel_df))

In [ ]:
clean_hotel_df.isnull().sum()

In [ ]:
clean_hotel_df.head(10)

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig